# SModelS – a tool for interpreting simplified-model results from the LHC

 <img src=https://smodels.github.io/images/banner720.png />

Jan Heisig and Andre Lessa, for the SModelS collaboration.

Re-Interpretation Workshop 2021, February 2021.

## Preparatory step -- installation:

### Check the installation instructions [here](https://indico.cern.ch/event/982553/contributions/4220784/attachments/2184008/3700911/smodels-installation.pdf) or install via pip:

In [3]:
# smodels is registered at the python packaging index
!pip3 install --user smodels==2.0.0b0

     |████████████████████████████████| 4.8 MB 19.6 MB/s eta 0:00:01
  Created wheel for smodels: filename=smodels-2.0.0b0-py3-none-any.whl size=9367384 sha256=db2c47890f7b699a4450ca93890e3d431ab606513cb91a053b955e03ddd85270
  Stored in directory: /home/lessa/.cache/pip/wheels/7f/a7/07/8c72fba842e11ab4a8cbd615e620fd040e9d9f72753aaa23d0
Successfully built smodels


### Check installation:

In [5]:
!runSModelS.py --help

usage: runSModelS.py [-h] -f FILENAME [-p PARAMETERFILE] [-o OUTPUTDIR] [-d]
                     [-t] [-C] [-V] [-c] [-v VERBOSE] [-T TIMEOUT]

Run SModelS over SLHA/LHE input files.

optional arguments:
  -h, --help            show this help message and exit
  -f FILENAME, --filename FILENAME
                        name of SLHA or LHE input file or a directory path
                        (required argument). If a directory is given, loop
                        over all files in the directory
  -p PARAMETERFILE, --parameterFile PARAMETERFILE
                        name of parameter file, where most options are defined
                        (optional argument). If not set, use all parameters
                        from smodels/etc/parameters_default.ini
  -o OUTPUTDIR, --outputDir OUTPUTDIR
                        name of output directory (optional argument). The
                        default folder is: ./results/
  -d, --development     if set, SModelS will run in development

In [7]:
!runSModelS.py -f ~/.local/smodels/inputFiles/slha/simplyGluino.slha

INFO in databaseObj.loadBinaryFile() in 212: loading binary db file /home/lessa/.cache/smodels/official200-beta.pcl format version 213
INFO in databaseObj.loadBinaryFile() in 219: Loaded database from /home/lessa/.cache/smodels/official200-beta.pcl in 0.7 secs.
INFO in modelTester.testPoints() in 285: Running SModelS for a single file
INFO in model.updateParticles() in 386: Loaded 62 BSM particles
INFO in model.updateParticles() in 386: Loaded 62 BSM particles
Input status: 1
Decomposition output status: 1 #decomposition was successful
# Input File: /home/lessa/.local/smodels/inputFiles/slha/simplyGluino.slha
# maxcond = 0.2
# minmassgap = 5
# ncpus = 1
# sigmacut = 0.01
# Database version: 2.0.0-beta
||                                                        ||
||                    Topologies Table                    ||
||                                                        ||
Topology:
Number of vertices: [1, 1] 
Number of vertex parts: [[2], [2]]
Total Global topology weight :
Sq

## First Step: Run SModelS for a single file

In [2]:
!runSModelS.py -p parameters.ini -f slhaFiles/wino_11010599.slha

## Second Step: Run SModelS for multiple files

In [ ]:
!runSModelS.py -p parameters_idm.ini -f slhaFiles/IDM_points -o results/IDM_output

## Third Step: plot the results

### Plot IDM results

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import pyslha
import matplotlib.pyplot as plt
import numpy as np
import imp,glob,os
cm = plt.cm.get_cmap('RdYlBu')

### Define path to SLHA and results folders

In [ ]:
slhaFolder = '../slhaFiles/IDM_points'
resultsFolder = '../results/IDM_output'

### Read SModelS results

In [ ]:
#Convert Experimental Results list to a dictionary
data = []
for f in glob.glob(resultsFolder+'/*.py'):
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    slhaFile = os.path.basename(smodelsDict['OutputStatus']['input file'])
    slhaFile = os.path.join(slhaFolder,slhaFile)
    #Read SLHA file (using pyslha):
    slhaData = pyslha.readSLHAFile(slhaFile)
    data.append((slhaData,smodelsDict))

In [ ]:
mHc = []
ctau = []
r = []
for slhaData,smodelsDict in data:
    if not 'ExptRes' in smodelsDict: #If no results were applicable, point is allowed
        rmax = 0.0
    else:
        rmax = smodelsDict['ExptRes'][0]['r'] #First result is the most constraining
    mass = slhaData.blocks['MASS'][37] #H+ mass
    width = slhaData.decays[37].totalwidth #H+ mass
    if width:    
        ct = 1.967e-16/width
    else:
        ct = 1e7
    
    mHc.append(mass)
    ctau.append(ct)
    r.append(rmax)

### Plot results:

In [ ]:
plt.figure(figsize=(8,6))
cp = plt.scatter(mHc,ctau, c=r, vmin=0., vmax=2., s=25, cmap=cm)
cb = plt.colorbar(cp)
plt.yscale('log')
plt.xlabel(r'$m_{H^{\pm}}$ (GeV)',fontsize=20)    
plt.ylabel(r'$c\tau$ (m)',fontsize=20)
cb.set_label(r'$r=\sigma_{th}/\sigma_{UL}$',fontsize=20)
plt.tight_layout()
plt.show()